DATA ANALYSIS


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("="*60)
print("HANGMAN CORPUS - EXPLORATORY DATA ANALYSIS (TRAINING ONLY)")
print("="*60)

# Load only the corpus (training set)
print("\n1. Loading Data...")
with open('Data/Data/corpus.txt', 'r', encoding='utf-8') as f:
    corpus_words = [word.strip().lower() for word in f.readlines() if word.strip()]

print(f"✓ Corpus size: {len(corpus_words)} words")

# DataFrame for corpus
corpus_df = pd.DataFrame({
    'word': corpus_words,
    'length': [len(word) for word in corpus_words],
    'unique_letters': [len(set(word)) for word in corpus_words]
})

print("\n" + "="*60)
print("2. BASIC STATISTICS")
print("="*60)

print("\nCorpus Statistics:")
print(corpus_df.describe())

print("\n" + "="*60)
print("3. WORD LENGTH ANALYSIS")
print("="*60)

length_dist = corpus_df['length'].value_counts().sort_index()
print("\nWord Length Distribution (Corpus):")
print(length_dist)

print(f"\nMost common word length: {corpus_df['length'].mode()[0]} letters")
print(f"Average word length: {corpus_df['length'].mean():.2f} letters")
print(f"Median word length: {corpus_df['length'].median():.0f} letters")
print(f"Shortest word: {corpus_df['length'].min()} letters")
print(f"Longest word: {corpus_df['length'].max()} letters")

print("\n" + "="*60)
print("4. LETTER FREQUENCY ANALYSIS")
print("="*60)

all_letters = ''.join(corpus_words)
letter_freq = Counter(all_letters)
total_letters = sum(letter_freq.values())

print("\nTop 15 Most Frequent Letters in Corpus:")
for letter, count in letter_freq.most_common(15):
    percentage = (count / total_letters) * 100
    print(f"  {letter}: {count:,} ({percentage:.2f}%)")

print("\nLeast Common Letters in Corpus:")
for letter, count in letter_freq.most_common()[-5:]:
    percentage = (count / total_letters) * 100
    print(f"  {letter}: {count:,} ({percentage:.2f}%)")

print("\n" + "="*60)
print("5. POSITION-BASED LETTER ANALYSIS")
print("="*60)

starting_letters = Counter([word[0] for word in corpus_words])
print("\nTop 10 Starting Letters:")
for letter, count in starting_letters.most_common(10):
    percentage = (count / len(corpus_words)) * 100
    print(f"  {letter}: {count} words ({percentage:.2f}%)")

ending_letters = Counter([word[-1] for word in corpus_words])
print("\nTop 10 Ending Letters:")
for letter, count in ending_letters.most_common(10):
    percentage = (count / len(corpus_words)) * 100
    print(f"  {letter}: {count} words ({percentage:.2f}%)")

print("\n" + "="*60)
print("6. UNIQUE LETTERS PER WORD")
print("="*60)

print(f"\nAverage unique letters per word: {corpus_df['unique_letters'].mean():.2f}")
print(f"Median unique letters per word: {corpus_df['unique_letters'].median():.0f}")

print("\nDistribution of Unique Letters:")
unique_dist = corpus_df['unique_letters'].value_counts().sort_index()
for num, count in unique_dist.items():
    percentage = (count / len(corpus_words)) * 100
    print(f"  {num} unique letters: {count} words ({percentage:.2f}%)")

print("\n" + "="*60)
print("7. VOWEL vs CONSONANT ANALYSIS")
print("="*60)

vowels = set('aeiou')
consonants = set('bcdfghjklmnpqrstvwxyz')

def count_vowels(word):
    return sum(1 for c in word if c in vowels)

def count_consonants(word):
    return sum(1 for c in word if c in consonants)

corpus_df['vowels'] = corpus_df['word'].apply(count_vowels)
corpus_df['consonants'] = corpus_df['word'].apply(count_consonants)
corpus_df['vowel_ratio'] = corpus_df['vowels'] / corpus_df['length']

print(f"\nAverage vowels per word: {corpus_df['vowels'].mean():.2f}")
print(f"Average consonants per word: {corpus_df['consonants'].mean():.2f}")
print(f"Average vowel ratio: {corpus_df['vowel_ratio'].mean():.2%}")

print("\nVowel Count Distribution:")
vowel_dist = corpus_df['vowels'].value_counts().sort_index()
for num, count in vowel_dist.items():
    percentage = (count / len(corpus_words)) * 100
    print(f"  {num} vowels: {count} words ({percentage:.2f}%)")

print("\n" + "="*60)
print("8. BIGRAM ANALYSIS")
print("="*60)

bigrams = []
for word in corpus_words:
    for i in range(len(word) - 1):
        bigrams.append(word[i:i+2])

bigram_freq = Counter(bigrams)
print("\nTop 20 Most Common Bigrams:")
for bigram, count in bigram_freq.most_common(20):
    percentage = (count / len(bigrams)) * 100
    print(f"  '{bigram}': {count} occurrences ({percentage:.2f}%)")

print("\n" + "="*60)
print("9. REPEATED LETTERS ANALYSIS")
print("="*60)

def has_repeated_letters(word):
    for i in range(len(word) - 1):
        if word[i] == word[i+1]:
            return True
    return False

words_with_repeats = sum(1 for word in corpus_words if has_repeated_letters(word))
print(f"\nWords with repeated letters: {words_with_repeats} ({words_with_repeats/len(corpus_words)*100:.2f}%)")

repeated_pairs = []
for word in corpus_words:
    for i in range(len(word) - 1):
        if word[i] == word[i+1]:
            repeated_pairs.append(word[i])

repeated_freq = Counter(repeated_pairs)
print("\nMost Common Repeated Letters:")
for letter, count in repeated_freq.most_common(10):
    print(f"  {letter}{letter}: {count} occurrences")

print("\n" + "="*60)
print("10. WORD COMPLEXITY ANALYSIS")
print("="*60)

corpus_df['uniqueness_ratio'] = corpus_df['unique_letters'] / corpus_df['length']

print("\nWords with ALL unique letters (no repeats):")
all_unique = corpus_df[corpus_df['uniqueness_ratio'] == 1.0]
print(f"Count: {len(all_unique)} words ({len(all_unique)/len(corpus_words)*100:.2f}%)")
print(f"Examples: {all_unique['word'].head(10).tolist()}")

print("\nWords with HIGH letter repetition (uniqueness < 0.5):")
high_repeat = corpus_df[corpus_df['uniqueness_ratio'] < 0.5]
print(f"Count: {len(high_repeat)} words ({len(high_repeat)/len(corpus_words)*100:.2f}%)")
print(f"Examples: {high_repeat['word'].head(10).tolist()}")

print("\n" + "="*60)
print("11. SAMPLE WORDS BY LENGTH")
print("="*60)

for length in sorted(corpus_df['length'].unique())[:10]:
    sample_words = corpus_df[corpus_df['length'] == length]['word'].head(5).tolist()
    print(f"\nLength {length}: {', '.join(sample_words)}")

print("\n" + "="*60)
print("13. KEY INSIGHTS FOR HANGMAN STRATEGY (TRAINING SET ONLY)")
print("="*60)

print("\n✓ MOST VALUABLE FIRST GUESSES (by frequency):")
top_letters = [letter for letter, _ in letter_freq.most_common(10)]
print(f"  Recommended order: {', '.join(top_letters)}")

print("\n✓ POSITION STRATEGY:")
print(f"  - Best starting letter guesses: {', '.join([l for l, _ in starting_letters.most_common(5)])}")
print(f"  - Best ending letter guesses: {', '.join([l for l, _ in ending_letters.most_common(5)])}")

print("\n✓ COMMON PATTERNS:")
print(f"  - Most common bigrams: {', '.join([b for b, _ in bigram_freq.most_common(5)])}")
print(f"  - Most common repeated letters: {', '.join([l for l, _ in repeated_freq.most_common(5)])}")

print("\n✓ WORD LENGTH STRATEGY:")
most_common_lengths = corpus_df['length'].value_counts().head(3)
print("  Most common word lengths:")
for length, count in most_common_lengths.items():
    print(f"    - {length} letters: {count} words ({count/len(corpus_words)*100:.2f}%)")

print("\n" + "="*60)
print("EDA COMPLETE! (TEST DATA NEVER TOUCHED)")
print("="*60)

# Optionally save only corpus/training summary to CSV
summary_df = pd.DataFrame({
    'Metric': ['Total Words', 'Unique Words', 'Avg Length', 'Most Common Length', 
               'Total Letters', 'Unique Letters Used'],
    'Corpus': [len(corpus_words), len(set(corpus_words)), 
               f"{corpus_df['length'].mean():.2f}", corpus_df['length'].mode()[0],
               total_letters, len(letter_freq)]
})
summary_df.to_csv('corpus_eda_summary.csv', index=False)
print("\n✓ Training summary saved to: corpus_eda_summary.csv")


HANGMAN CORPUS - EXPLORATORY DATA ANALYSIS (TRAINING ONLY)

1. Loading Data...
✓ Corpus size: 50000 words

2. BASIC STATISTICS

Corpus Statistics:
             length  unique_letters
count  50000.000000    50000.000000
mean       9.497460        7.513740
std        2.957487        1.972828
min        1.000000        1.000000
25%        7.000000        6.000000
50%        9.000000        8.000000
75%       11.000000        9.000000
max       24.000000       15.000000

3. WORD LENGTH ANALYSIS

Word Length Distribution (Corpus):
length
1       46
2       84
3      388
4     1169
5     2340
6     3755
7     5111
8     6348
9     6808
10    6465
11    5452
12    4292
13    3094
14    2019
15    1226
16     698
17     375
18     174
19      88
20      40
21      16
22       8
23       3
24       1
Name: count, dtype: int64

Most common word length: 9 letters
Average word length: 9.50 letters
Median word length: 9 letters
Shortest word: 1 letters
Longest word: 24 letters

4. LETTER FREQUENCY 

In [2]:
# ==================== COMPLETE HANGMAN SOLVER ====================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import pickle
import random
import re

# ==================== STATISTICAL HMM ====================
class StatisticalHangmanHMM:
    def __init__(self):
        self.words = []
        self.global_freq = Counter()
        self.length_freq = {}
        self.first_letter_freq = Counter()
        self.second_letter_freq = Counter()
        self.last_letter_freq = Counter()
        self.second_last_freq = Counter()
        self.position_bins = {i: Counter() for i in range(10)}
        self.after_letter = defaultdict(Counter)
        self.before_letter = defaultdict(Counter)
        self.letter_pairs = Counter()
        self.vowels = set('aeiou')
        self.consonants = set('bcdfghjklmnpqrstvwxyz')
        self.vowel_positions = defaultdict(Counter)
        self.common_order = ['e', 'a', 'i', 'o', 'r', 'n', 't', 's', 'l', 'c', 'u', 'd', 'p', 'm', 'h']
        
    def train(self, corpus_path):
        print("Training Statistical HMM...")
        with open(corpus_path, 'r', encoding='utf-8') as f:
            self.words = [word.strip().lower() for word in f.readlines() if word.strip()]
        
        print(f"Analyzing {len(self.words)} words...")
        
        for word in self.words:
            wlen = len(word)
            
            # Global frequency
            for char in word:
                self.global_freq[char] += 1
            
            # Length-specific frequency
            if wlen not in self.length_freq:
                self.length_freq[wlen] = Counter()
            for char in word:
                self.length_freq[wlen][char] += 1
            
            # Fixed positions
            if wlen >= 1:
                self.first_letter_freq[word[0]] += 1
            if wlen >= 2:
                self.second_letter_freq[word[1]] += 1
                self.last_letter_freq[word[-1]] += 1
            if wlen >= 3:
                self.second_last_freq[word[-2]] += 1
            
            # Relative positions
            for i, char in enumerate(word):
                rel_pos = int((i / wlen) * 10) if wlen > 1 else 0
                rel_pos = min(rel_pos, 9)
                self.position_bins[rel_pos][char] += 1
            
            # Conditional probabilities
            for i in range(len(word) - 1):
                curr, nxt = word[i], word[i+1]
                self.after_letter[curr][nxt] += 1
                self.before_letter[nxt][curr] += 1
                self.letter_pairs[(curr, nxt)] += 1
            
            # Vowel positions
            for i, char in enumerate(word):
                if char in self.vowels:
                    rel_pos = int((i / wlen) * 10) if wlen > 1 else 0
                    rel_pos = min(rel_pos, 9)
                    self.vowel_positions[rel_pos][char] += 1
        
        # Normalize everything
        self._normalize()
        print("HMM training complete!")
    
    def _normalize(self):
        total = sum(self.global_freq.values())
        self.global_freq = {k: v/total for k, v in self.global_freq.items()}
        
        for length in self.length_freq:
            total = sum(self.length_freq[length].values())
            self.length_freq[length] = {k: v/total for k, v in self.length_freq[length].items()}
        
        for counter in [self.first_letter_freq, self.second_letter_freq, 
                       self.last_letter_freq, self.second_last_freq]:
            total = sum(counter.values())
            if total > 0:
                for k in list(counter.keys()):
                    counter[k] = counter[k] / total
        
        for i in range(10):
            total = sum(self.position_bins[i].values())
            if total > 0:
                self.position_bins[i] = {k: v/total for k, v in self.position_bins[i].items()}
        
        for letter in self.after_letter:
            total = sum(self.after_letter[letter].values())
            if total > 0:
                self.after_letter[letter] = {k: v/total for k, v in self.after_letter[letter].items()}
        
        for letter in self.before_letter:
            total = sum(self.before_letter[letter].values())
            if total > 0:
                self.before_letter[letter] = {k: v/total for k, v in self.before_letter[letter].items()}
        
        for pos in self.vowel_positions:
            total = sum(self.vowel_positions[pos].values())
            if total > 0:
                self.vowel_positions[pos] = {k: v/total for k, v in self.vowel_positions[pos].items()}
    
    def predict_letter_probabilities(self, pattern, guessed_letters, lives_remaining):
        wlen = len(pattern)
        scores = Counter()
        
        weights = {
            'global': 0.20,
            'length': 0.25,
            'position': 0.25,
            'context': 0.30
        }
        
        # 1. Global frequency baseline
        for letter in self.common_order:
            if letter not in guessed_letters:
                freq = self.global_freq.get(letter, 0.001)
                scores[letter] += weights['global'] * freq
        
        # 2. Length-specific frequency
        if wlen in self.length_freq:
            for letter, prob in self.length_freq[wlen].items():
                if letter not in guessed_letters:
                    scores[letter] += weights['length'] * prob
        
        # 3. Position-based predictions
        blank_positions = [i for i, c in enumerate(pattern) if c == '_']
        
        for pos in blank_positions:
            if pos == 0:
                for letter, prob in self.first_letter_freq.items():
                    if letter not in guessed_letters:
                        scores[letter] += weights['position'] * prob / len(blank_positions)
            elif pos == 1 and wlen >= 2:
                for letter, prob in self.second_letter_freq.items():
                    if letter not in guessed_letters:
                        scores[letter] += weights['position'] * prob / len(blank_positions)
            elif pos == wlen - 1:
                for letter, prob in self.last_letter_freq.items():
                    if letter not in guessed_letters:
                        scores[letter] += weights['position'] * prob / len(blank_positions)
            elif pos == wlen - 2 and wlen >= 3:
                for letter, prob in self.second_last_freq.items():
                    if letter not in guessed_letters:
                        scores[letter] += weights['position'] * prob / len(blank_positions)
            
            rel_pos = int((pos / wlen) * 10) if wlen > 1 else 0
            rel_pos = min(rel_pos, 9)
            for letter, prob in self.position_bins[rel_pos].items():
                if letter not in guessed_letters:
                    scores[letter] += weights['position'] * prob * 0.5 / len(blank_positions)
        
        # 4. Context-based
        for i, char in enumerate(pattern):
            if char != '_':
                if i > 0 and pattern[i-1] == '_':
                    for prev_letter, prob in self.before_letter[char].items():
                        if prev_letter not in guessed_letters:
                            scores[prev_letter] += weights['context'] * prob
                
                if i < wlen - 1 and pattern[i+1] == '_':
                    for next_letter, prob in self.after_letter[char].items():
                        if next_letter not in guessed_letters:
                            scores[next_letter] += weights['context'] * prob
        
        # 5. Vowel pattern heuristic
        revealed_letters = [c for c in pattern if c != '_']
        vowel_count = sum(1 for c in revealed_letters if c in self.vowels)
        consonant_count = len(revealed_letters) - vowel_count
        
        if consonant_count > vowel_count + 2:
            for letter in self.vowels:
                if letter not in guessed_letters:
                    scores[letter] *= 1.5
        
        if vowel_count > consonant_count:
            for letter in self.consonants:
                if letter not in guessed_letters:
                    scores[letter] *= 1.2
        
        total = sum(scores.values())
        if total > 0:
            scores = {k: v/total for k, v in scores.items()}
        else:
            scores = {letter: 1.0/(i+1) for i, letter in enumerate(self.common_order) 
                     if letter not in guessed_letters}
            total = sum(scores.values())
            scores = {k: v/total for k, v in scores.items()}
        
        return dict(scores)
    
    def save_model(self, filepath):
        with open(filepath, 'wb') as f:
            pickle.dump(self, f)
        print(f"Model saved to {filepath}")
    
    @staticmethod
    def load_model(filepath):
        with open(filepath, 'rb') as f:
            return pickle.load(f)


In [3]:
# ==================== RL AGENT ====================
class GeneralStrategyRLAgent:
    def __init__(self, hmm_model):
        self.hmm = hmm_model
        self.strategy_weights = {}
        self.letter_bias = np.zeros(26)
        self.learning_rate = 0.005
        self.epsilon = 0.2
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.05
        self.gamma = 0.9

    def get_state_features(self, pattern, guessed_letters, lives_remaining):
        wlen = len(pattern)
        revealed = sum(1 for c in pattern if c != '_')
        progress = revealed / wlen if wlen > 0 else 0
        length_bucket = min(wlen // 3, 4)
        progress_bucket = int(progress * 4)
        lives_bucket = min(lives_remaining // 2, 2)
        return (length_bucket, progress_bucket, lives_bucket)

    def get_strategy_weights(self, state):
        if state not in self.strategy_weights:
            self.strategy_weights[state] = np.array([0.20, 0.25, 0.25, 0.30])
        return self.strategy_weights[state]

    def select_action(self, pattern, guessed_letters, hmm_probs, lives_remaining, epsilon=None):
        if epsilon is None:
            epsilon = self.epsilon
        available_letters = [chr(ord('a') + i) for i in range(26) if chr(ord('a') + i) not in guessed_letters]
        if not available_letters:
            return 'e'

        adjusted_probs = {}
        for letter in available_letters:
            hmm_prob = hmm_probs.get(letter, 0.001)
            letter_idx = ord(letter) - ord('a')
            bias = self.letter_bias[letter_idx]
            adjusted_prob = hmm_prob * (1.0 + bias * 0.2)
            adjusted_probs[letter] = max(adjusted_prob, 0.001)
        
        total = sum(adjusted_probs.values())
        for letter in adjusted_probs:
            adjusted_probs[letter] /= total

        if np.random.random() < epsilon:
            letters = list(adjusted_probs.keys())
            probs = np.array([adjusted_probs[l] for l in letters])
            probs = probs / probs.sum()
            letter = np.random.choice(letters, p=probs)
        else:
            letter = max(adjusted_probs.items(), key=lambda x:x[1])[0]
        return letter

    def update(self, state_features, letter, reward, next_state_features, done):
        letter_idx = ord(letter) - ord('a')
        if reward > 0:
            self.letter_bias[letter_idx] += self.learning_rate * 0.5
        elif reward < 0:
            self.letter_bias[letter_idx] -= self.learning_rate * 0.3
        self.letter_bias = np.clip(self.letter_bias, -2.0, 2.0)

    def decay_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save_model(self, filepath):
        data = {
            'letter_bias': self.letter_bias,
            'strategy_weights': self.strategy_weights,
            'epsilon': self.epsilon,
        }
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)
        print(f"RL Agent saved to {filepath}")

    def load_model(self, filepath):
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
        self.letter_bias = data['letter_bias']
        self.strategy_weights = data['strategy_weights']
        self.epsilon = data['epsilon']
        print(f"RL Agent loaded from {filepath}")

In [4]:
# ==================== ENVIRONMENT ====================
class HangmanEnvironment:
    def __init__(self, word, hmm_model):
        self.word = word.lower()
        self.hmm = hmm_model
        self.guessed_letters = set()
        self.lives_remaining = 6
        self.current_pattern = ''.join('_' for _ in self.word)
        self.gameover = False
        self.won = False

    def get_state_features(self):
        wlen = len(self.current_pattern)
        revealed = sum(1 for c in self.current_pattern if c != '_')
        progress = revealed / wlen if wlen > 0 else 0
        length_bucket = min(wlen // 3, 4)
        progress_bucket = int(progress * 4)
        lives_bucket = min(self.lives_remaining // 2, 2)
        return (length_bucket, progress_bucket, lives_bucket)

    def step(self, letter):
        if self.gameover:
            return 0, True, {'error': 'Game already over'}
        if letter in self.guessed_letters:
            return -10.0, False, {'type': 'repeated', 'correct': False}
        self.guessed_letters.add(letter)
        if letter in self.word:
            self.current_pattern = ''.join(
                c if c in self.guessed_letters else '_' for c in self.word
            )
            if '_' not in self.current_pattern:
                self.gameover = True
                self.won = True
                return 100.0, True, {'type': 'win', 'correct': True}
            else:
                num_revealed = self.current_pattern.count(letter)
                return 10.0 * num_revealed + 2.0, False, {'type': 'correct', 'correct': True}
        else:
            self.lives_remaining -= 1
            if self.lives_remaining == 0:
                self.gameover = True
                self.won = False
                return -100.0, True, {'type': 'lose', 'correct': False}
            else:
                penalty = -10.0 - (6 - self.lives_remaining) * 3.0
                return penalty, False, {'type': 'wrong', 'correct': False}

In [5]:
# ==================== RL TRAINER ====================
class RLTrainer:
    def __init__(self, hmm_model, words_list):
        self.hmm = hmm_model
        self.words = words_list
        self.agent = GeneralStrategyRLAgent(hmm_model)

    def train(self, num_episodes=2000, words_per_episode=100):
        print("="*80)
        print(f"RL TRAINING {num_episodes} EPISODES")
        print(f"Words per episode: {words_per_episode}")
        print(f"Total training games: {num_episodes * words_per_episode:,}")
        print("="*80)
        
        # Track performance metrics
        episode_metrics = []
        
        for episode in range(num_episodes):
            episode_wins = 0
            episode_games = 0
            episode_reward = 0
            episode_correct_guesses = 0
            episode_wrong_guesses = 0
            
            sampled_words = random.sample(self.words, min(words_per_episode, len(self.words)))
            for word in sampled_words:
                env = HangmanEnvironment(word, self.hmm)
                done = False
                while not done:
                    state = env.get_state_features()
                    hmm_probs = self.hmm.predict_letter_probabilities(
                        env.current_pattern,
                        env.guessed_letters,
                        env.lives_remaining,
                    )
                    letter = self.agent.select_action(
                        env.current_pattern,
                        env.guessed_letters,
                        hmm_probs,
                        env.lives_remaining
                    )
                    reward, done, info = env.step(letter)
                    next_state = env.get_state_features()
                    self.agent.update(state, letter, reward, next_state, done)
                    episode_reward += reward
                    
                    # Track guess statistics
                    if info['type'] == 'correct':
                        episode_correct_guesses += 1
                    elif info['type'] == 'wrong':
                        episode_wrong_guesses += 1
                        
                    if env.won:
                        episode_wins += 1
                episode_games += 1
            
            self.agent.decay_epsilon()
            
            # Calculate metrics for this episode
            win_rate = episode_wins / episode_games * 100 if episode_games > 0 else 0
            avg_reward = episode_reward / episode_games if episode_games > 0 else 0
            accuracy = episode_correct_guesses / (episode_correct_guesses + episode_wrong_guesses) * 100 if (episode_correct_guesses + episode_wrong_guesses) > 0 else 0
            
            episode_metrics.append({
                'episode': episode + 1,
                'win_rate': win_rate,
                'avg_reward': avg_reward,
                'accuracy': accuracy,
                'epsilon': self.agent.epsilon
            })
            
            # Print progress every 100 episodes
            if (episode + 1) % 100 == 0 or (episode + 1) <= 10:
                print(f"Episode {episode+1:5d}: "
                      f"Win Rate {win_rate:6.2f}% | "
                      f"Avg Reward {avg_reward:7.1f} | "
                      f"Accuracy {accuracy:6.2f}% | "
                      f"Epsilon {self.agent.epsilon:.4f}")
            
            # More detailed report every 500 episodes
            if (episode + 1) % 500 == 0:
                # Calculate rolling averages
                recent_metrics = episode_metrics[-100:]  # Last 100 episodes
                recent_win_rate = np.mean([m['win_rate'] for m in recent_metrics])
                recent_accuracy = np.mean([m['accuracy'] for m in recent_metrics])
                
                print("-" * 80)
                print(f"PROGRESS REPORT - Episode {episode+1}")
                print(f"Recent Performance (last 100 episodes):")
                print(f"  Average Win Rate: {recent_win_rate:.2f}%")
                print(f"  Average Accuracy: {recent_accuracy:.2f}%")
                print(f"  Current Epsilon:  {self.agent.epsilon:.4f}")
                print("-" * 80)
        
        # Final training summary
        print("="*80)
        print("RL TRAINING COMPLETE!")
        print("="*80)
        
        # Calculate overall statistics
        final_100_metrics = episode_metrics[-100:]
        final_win_rate = np.mean([m['win_rate'] for m in final_100_metrics])
        final_accuracy = np.mean([m['accuracy'] for m in final_100_metrics])
        
        print(f"FINAL PERFORMANCE (last 100 episodes):")
        print(f"  Win Rate:  {final_win_rate:.2f}%")
        print(f"  Accuracy:  {final_accuracy:.2f}%")
        print(f"  Epsilon:   {self.agent.epsilon:.4f}")
        print("="*80)
        
        return self.agent, episode_metrics


In [6]:
# ==================== EVALUATOR ====================
class FinalEvaluator:
    def __init__(self, hmm_model, agent, test_words):
        self.hmm = hmm_model
        self.agent = agent
        self.test_words = test_words
        
    def evaluate(self, num_games=None):
        if num_games is None:
            num_games = len(self.test_words)
        
        test_words = self.test_words[:num_games]
        
        wins = 0
        total_wrong = 0
        total_repeated = 0
        total_guesses = 0
        total_correct_guesses = 0
        
        print(f"\n{'='*80}")
        print(f"EVALUATING ON {num_games} TEST GAMES")
        print(f"{'='*80}")
        
        for i, word in enumerate(test_words):
            env = HangmanEnvironment(word, self.hmm)
            done = False
            
            while not done:
                hmm_probs = self.hmm.predict_letter_probabilities(
                    env.current_pattern,
                    env.guessed_letters,
                    env.lives_remaining
                )
                
                letter = self.agent.select_action(
                    env.current_pattern,
                    env.guessed_letters,
                    hmm_probs,
                    env.lives_remaining,
                    epsilon=0.0  # Greedy at test time
                )
                
                reward, done, info = env.step(letter)
                total_guesses += 1
                
                if info['type'] == 'wrong':
                    total_wrong += 1
                elif info['type'] == 'repeated':
                    total_repeated += 1
                elif info['type'] == 'correct':
                    total_correct_guesses += 1
            
            if env.won:
                wins += 1
            
            if (i + 1) % 500 == 0 or (i + 1) == num_games:
                current_wr = (wins / (i + 1)) * 100
                print(f"  Progress: {i + 1:5d}/{num_games} | Win Rate: {current_wr:6.2f}%")
        
        success_rate = wins / num_games
        test_accuracy = total_correct_guesses / total_guesses * 100 if total_guesses > 0 else 0
        final_score = wins - (total_wrong * 5) - (total_repeated * 2)
        
        print(f"\n{'='*80}")
        print("FINAL TEST RESULTS")
        print(f"{'='*80}")
        print(f"Total Games:          {num_games}")
        print(f"Wins:                 {wins} ({wins/num_games*100:.2f}%)")
        print(f"Losses:               {num_games - wins} ({(num_games-wins)/num_games*100:.2f}%)")
        print(f"Total Guesses:        {total_guesses}")
        print(f"Correct Guesses:      {total_correct_guesses} ({test_accuracy:.2f}% accuracy)")
        print(f"Wrong Guesses:        {total_wrong} (avg: {total_wrong/num_games:.2f} per game)")
        print(f"Repeated Guesses:     {total_repeated}")
        print(f"\nScore Breakdown:")
        print(f"  Success Points:     +{wins}")
        print(f"  Wrong Penalty:      -{total_wrong * 5}")
        print(f"  Repeated Penalty:   -{total_repeated * 2}")
        print(f"\n  FINAL SCORE:        {final_score}")
        print(f"{'='*80}")
        
        return wins, total_wrong, total_repeated, final_score, test_accuracy

In [7]:
# ==================== DEMO FUNCTION ====================
def demo_agent_play(agent, hmm_model, words):
    print("="*80)
    print("HANGMAN RL AGENT STEP-BY-STEP DEMO")
    print("="*80)

    for word in words:
        print(f"\nStarting new game for word: '{word}'")
        env = HangmanEnvironment(word, hmm_model)
        done = False
        step_num = 1

        # Header for each step
        print(f"{'Step':<6}{'Guess':<8}{'Pattern':<20}{'Lives Left':<12}{'Reward':<8}{'Game Status':<15}")
        print("-"*80)

        while not done:
            # Obtain HMM probabilities to assist agent's guess
            hmm_probs = hmm_model.predict_letter_probabilities(
                env.current_pattern, env.guessed_letters, env.lives_remaining
            )

            # Agent chooses next letter (deterministic selection for demo)
            letter = agent.select_action(
                env.current_pattern, env.guessed_letters, hmm_probs, env.lives_remaining,
                epsilon=0.0
            )

            # Perform a step in environment
            reward, done, info = env.step(letter)

            # Display the step details neatly
            game_status = "Won!" if env.won else ("Lost!" if done else "Ongoing")
            print(f"{step_num:<6}{letter:<8}{env.current_pattern:<20}{env.lives_remaining:<12}{reward:<8.1f}{game_status:<15}")

            step_num += 1

        print(f"\nGame complete for word '{word}': {'SUCCESS' if env.won else 'FAILURE'} in {step_num-1} steps")
        print("="*80)


In [8]:
# ==================== MAIN EXECUTION ====================
if __name__ == "__main__":
    print("="*80)
    print("HANGMAN SOLVER: Statistical HMM + RL")
    print("Reduced Training: 2,000 Episodes")
    print("(Anti-overfitting design)")
    print("="*80)
    
    # Train HMM
    try:
        print("\nLoading existing HMM...")
        hmm = StatisticalHangmanHMM.load_model('statistical_hmm.pkl')
    except:
        print("\nTraining new HMM...")
        hmm = StatisticalHangmanHMM()
        hmm.train('Data/Data/corpus.txt')
        hmm.save_model('statistical_hmm.pkl')
    
    # Load corpus for training
    print("\nLoading training data...")
    with open('Data/Data/corpus.txt', 'r') as f:
        corpus_words = [w.strip().lower() for w in f.readlines() if w.strip()]
    
    # Train RL agent with reduced episodes
    print(f"\nTraining RL agent for 2,000 episodes...")
    trainer = RLTrainer(hmm, corpus_words)
    agent, training_metrics = trainer.train(num_episodes=2000, words_per_episode=100)
    agent.save_model('rl_agent_2000.pkl')
    
    # Load test set
    print("\nLoading test data...")
    with open('Data/Data/test.txt', 'r') as f:
        test_words = [w.strip().lower() for w in f.readlines() if w.strip()]
    
    # Evaluate
    print(f"\nEvaluating on test set...")
    evaluator = FinalEvaluator(hmm, agent, test_words)
    wins, wrong, repeated, score, accuracy = evaluator.evaluate()
    
    # Demo with sample words
    print(f"\nRunning demo with sample words...")
    example_words = ["python", "perfume", "aditya", "hangman", "challenge"]
    demo_agent_play(agent, hmm, example_words)
    
    print("\n" + "="*80)
    print("TRAINING AND EVALUATION COMPLETE!")
    print("="*80)
    
    # Save training metrics for analysis
    metrics_df = pd.DataFrame(training_metrics)
    metrics_df.to_csv('training_metrics_2000.csv', index=False)
    print(f"Training metrics saved to: training_metrics_2000.csv")

HANGMAN SOLVER: Statistical HMM + RL
Reduced Training: 2,000 Episodes
(Anti-overfitting design)

Loading existing HMM...

Loading training data...

Training RL agent for 2,000 episodes...
RL TRAINING 2000 EPISODES
Words per episode: 100
Total training games: 200,000
Episode     1: Win Rate  21.00% | Avg Reward   -68.4 | Accuracy  52.40% | Epsilon 0.1990
Episode     2: Win Rate  26.00% | Avg Reward   -50.6 | Accuracy  54.39% | Epsilon 0.1980
Episode     3: Win Rate  18.00% | Avg Reward   -73.0 | Accuracy  52.47% | Epsilon 0.1970
Episode     4: Win Rate  24.00% | Avg Reward   -60.0 | Accuracy  53.61% | Epsilon 0.1960
Episode     5: Win Rate  26.00% | Avg Reward   -51.7 | Accuracy  54.12% | Epsilon 0.1950
Episode     6: Win Rate  22.00% | Avg Reward   -62.7 | Accuracy  53.04% | Epsilon 0.1941
Episode     7: Win Rate  16.00% | Avg Reward   -75.3 | Accuracy  52.35% | Epsilon 0.1931
Episode     8: Win Rate  23.00% | Avg Reward   -57.0 | Accuracy  54.19% | Epsilon 0.1921
Episode     9: Win Ra